In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from helpers import *
from matplotlib import pyplot as plt
import numpy as np
from experiment_mnist import *
from experiment_constant import *
from experiment_random import *
from scipy.special import expit
import pandas as pd
import seaborn as sns

Using TensorFlow backend.


Testing equations:
1. $\mathbb{E}_\xi\Delta_L=p\sum\left(y(\vec{x}-x_i)-y(\vec{x})\right)+O(p^2)$
2. $\mathbb{E}_\xi\Delta_L=-p\nabla_x y_L(x)x+O(p^2x_i^2)$
3. $Var_\xi\Delta_L=p\sum\left(y(\vec{x}-x_i)-y(\vec{x})\right)^2+O(p^2)$
4. $Var_\xi\Delta_L=p(\nabla_x y_L(x))^2x^2+O(p^2x_i^2)$
5. $Var_\xi\Delta_L=pW^2_LK^2_{L-1}\odot W^2_{L-1}...W_2^2K_1^2\odot W_1^2x^2+O(p^2x_i^2w^2)$

Testing theorems:
1. $\mathbb{E}_{x}\mathbb{E}_{\xi}\Delta_L=O(p^2x_i^2)$
2. 	$\sum\limits_{i,j}\frac{\partial y_k}{\partial w_{ij}}w_{ij}=\sum\limits_{j}\frac{\partial y_k}{\partial x_j}x_j
    = -\frac{1}{p}(\mathbb{E}_\xi\Delta_L^k+O(p^2))$

In [2]:
# pfailure for the first layer
p = 1e-2

In [31]:
# run with random weights and multiple inputs
exp = RandomExperiment([50, 50, 50, 50, 50, 2], [p, 0, 0, 0], 1, activation = 'relu', std_weight = 3, mean_weight = 0.1)

In [32]:
# get input and calculate error
x = np.random.randn(50)

In [33]:
errors = exp.get_error(x, repetitions = 10000)
print(np.mean(errors, axis = 0), np.std(errors, axis = 0))

[-2.6607046 -2.3084924] [4.9939656 4.3327928]


Testing $\mathbb{E}_\xi\Delta_L=p\sum\left(y(\vec{x}-x_i)-y(\vec{x})\right)+O(p^2)$
and $\mathbb{E}_\xi\Delta_L=-p\nabla_x y_L(x)x+O(p^2x_i^2)$

In [34]:
print(exp.get_exact_error_v3_better(x))
print(exp.get_exact_error_v3(x))
print(exp.get_exact_error_v3_tf(x))

[[-2.6334872 -2.2849035]]
[[-2.63348504]
 [-2.28490954]]
[[-2.633485  -2.2849097]]


Testing
3. $Var_\xi\Delta_L=p\sum\left(y(\vec{x}-x_i)-y(\vec{x})\right)^2+O(p^2)$
4. $Var_\xi\Delta_L=p(\nabla_x y_L(x))^2x^2+O(p^2x_i^2)$
5. $Var_\xi\Delta_L=pW^2_LK^2_{L-1}\odot W^2_{L-1}...W_2^2K_1^2\odot W_1^2x^2+O(p^2x_i^2w^2)$

In [42]:
print(exp.get_exact_std_error_v3_better(x))
print(exp.get_exact_std_error_v3_tf(x))

[[4.9761105 4.3172855]]
[[4.9761086 4.317294 ]]
